## MNIST with CNN

### Setup Github Access
For accessing the package in Github, we need to set up the Github access token and actor. This is done by setting the `GITHUB_ACTOR` and `GITHUB_TOKEN` environment variables.

In [ ]:
val githubActor  = System.getenv("GITHUB_ACTOR")
val githubToken = System.getenv("GITHUB_TOKEN")

### Setup Kotlin Jupyter Dependencies

In [ ]:
USE {
    repositories {
        maven {
            url = "https://maven.pkg.github.com/sk-ai-net/skainet-jupyter"
            credentials {
                username = githubActor
                password = githubToken
            }
        }
        mavenCentral()
    }
    dependencies {
        implementation("sk.ai.net:kotlin-jupyter:0.0.7")
        implementation("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.10.2")
        val ktorVersion = "3.1.3"
        implementation("io.ktor:ktor-client-cio:$ktorVersion")
        implementation("io.ktor:ktor-client-core:$ktorVersion")
        implementation("io.ktor:ktor-client-content-negotiation:$ktorVersion")
        implementation("io.ktor:ktor-client-logging:$ktorVersion")
    }
}

### Importing Libraries

In [ ]:
// SKaiNET
import sk.ai.net.Tensor
import sk.ai.net.dsl.network
import sk.ai.net.nn.activations.ReLU
import sk.ai.net.nn.Module
import sk.ai.net.Shape
import sk.ai.net.impl.DoublesTensor


// SKaiNET - MNIST
import sk.ai.net.io.data.mnist.MNISTLoaderFactory
import sk.ai.net.io.data.mnist.MNISTConstants
import sk.ai.net.io.data.mnist.MNISTImage
import sk.ai.net.io.data.mnist.MNISTDataset


// Kotlin
import kotlin.random.Random
import kotlinx.coroutines.runBlocking

### Download MNIST dataset

In [ ]:
val (trainingData, testData) = runBlocking {
    MNISTLoaderFactory.create().let { loader ->
        loader.loadTrainingData() to loader.loadTestData()
    }
}

### CNN model

In [12]:
val mnistNetwork = network {
    sequential {
        stage("conv1") {
            conv2d {
                outChannels = 16
                kernelSize = 5
                stride = 1
                padding = 2
            }
            activation("relu", ReLU()::forward)
            maxPool2d {
                kernelSize = 2
                stride = 2
            }
        }
        stage("conv2") {
            conv2d {
                outChannels = 32
                kernelSize = 5
                stride = 1
                padding = 2
            }
            activation("relu", ReLU()::forward)
            maxPool2d {
                kernelSize = 2
                stride = 2
            }
        }
        stage("flatten") {
            flatten()
        }
        stage("dense") {
            dense {
                units = 128
            }
            activation("relu", ReLU()::forward)
        }
        stage("output") {
            dense {
                units = 10
            }
            activation("relu", ReLU()::forward)
        }
    }
}


In [ ]:
mnistNetwork.summary(Shape(1,0,28,28))

### Simple MLP model

In [13]:
val mlp = network {
    input(28*28)
    dense(128) {
        activation = ReLU()::invoke
    }
    dense(10)
}

In [15]:
mlp.summary(Shape(28*28))

+--------------+------------------------------------------------+---------+
| Layer (type) | Output Shape                                   | Param # |
+--------------+------------------------------------------------+---------+
| linear-1     | Shape: Dimensions = [128], Size (Volume) = 128 | 100480  |
+--------------+------------------------------------------------+---------+
| linear-3     | Shape: Dimensions = [10], Size (Volume) = 10   | 1290    |
+--------------+------------------------------------------------+---------+
